In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
# from catboost import CatBoostClassifier
# from catboost.utils import get_roc_curve
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.datasets import dump_svmlight_file
from sklearn.metrics import accuracy_score


In [18]:
random_state = 42

In [19]:
train_df = pd.read_csv("data/df_train.csv")

In [20]:
train_df.shape

(8712, 208)

In [21]:
train_df.target.value_counts()

0    8187
1     525
Name: target, dtype: int64

In [22]:
# create training and testing vars
X_train, X_val, y_train, y_val = train_test_split(train_df.drop(columns=["target"]), train_df["target"], test_size=0.2,stratify=train_df["target"], random_state=42)
print (X_train.shape, y_train.shape)
print (X_val.shape, y_val.shape)

(6969, 207) (6969,)
(1743, 207) (1743,)


In [23]:
print(X_train.shape)
print(X_val.shape)

(6969, 207)
(1743, 207)


### Xgboost classifier

In [77]:
model = XGBClassifier(max_depth=10,learning_rate=0.01, max_delta_step=1)

In [78]:
model.fit(X_train,y_train,eval_metric="auc")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=1,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.5, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [79]:
y_train_pred = model.predict(X_train)

In [80]:
accuracy = accuracy_score(y_train, y_train_pred)
print(accuracy)
# print("Train Accuracy: %.2f%%" % (accuracy * 100.0))

0.9410245372363323


In [81]:
y_val_pred = model.predict(X_val)
# y_val_pred = [round(value) for value in y_val]

In [82]:
accuracy = accuracy_score(y_val, y_val_pred)
print(accuracy)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

0.9409064830751578


In [54]:
test_df = pd.read_csv("data/df_test.csv")

In [55]:
test_df.head()

,signal_id,id_measurement,phase,target,mean,max,min,std,8,9,...,198,199,200,201,202,203,204,205,206,207
0,8712,2904,0,-1,-1.020250,28,-46,13.818361,15,15,...,8,9,10,11,12,12,13,14,13,15
1,8713,2904,1,-1,-0.185500,30,-25,13.962542,-20,-21,...,-20,-20,-21,-20,-20,-20,-21,-20,-21,-20
2,8714,2904,2,-1,-0.033500,31,-27,13.809811,3,1,...,7,7,6,6,7,6,5,6,3,4
3,8715,2905,0,-1,-0.014125,28,-29,14.104341,11,16,...,10,10,10,10,11,11,10,13,11,13
4,8716,2905,1,-1,-1.484625,29,-35,13.572325,1,5,...,6,6,7,5,5,4,3,5,4,5


In [56]:
X_test = test_df.drop(columns=["target"])
y_test = test_df["target"]

In [57]:
y_pred_test = model.predict(X_test)
len(y_pred_test)

20337

### Create Submission File

In [58]:
submission_df = pd.DataFrame()

In [59]:
y_pred_test.shape

(20337,)

In [60]:
submission_df["signal_id"] = test_df["signal_id"]
submission_df["target"] = y_pred_test.astype("int8")

In [61]:
type(submission_df["target"][0])

numpy.int8

In [62]:
submission_df.head(2)

,signal_id,target
0,8712,0
1,8713,0


In [63]:
submission_df.to_csv("XGB_LR_01.csv", index=False)